In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

OriginURL = r"https://sh.lianjia.com/chengjiao/"
Header ={
"Host": "sh.lianjia.com",
"Connection": "keep-alive",
"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",
"Accept": "image/webp,image/apng,image/*,*/*;q=0.8",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "zh-CN,zh;q=0.9"}


def get_block_list(url):
    _connection = requests.get(url, header=Header)
    _BS = BeautifulSoup(_connection.text, "lxml")
    _BlockListOrigin = _BS.find("div", class_="position").find_all("a", href=re.compile("/chengjiao/\w+/"), rel=False)
    _BlockList = ["https://sh.lianjia.com" + i["href"] for i in _BlockListOrigin]
    return _BlockList


def get_block_list_of_a_city(url):
    _BlockList = []
    _connection = requests.get(url, header=Header)
    _BS = BeautifulSoup(_connection.text, "lxml")
    _DistrictListOrigin = _BS.find_all("a", href=re.compile("/chengjiao/\w+/"), title=re.compile("上海"))
    _DistrictList = ["https://sh.lianjia.com" + i["href"] for i in _DistrictListOrigin]
    print("Fetched the {}".format(url))
    for i in _DistrictList:
        templist = get_block_list(i)
        _BlockList = _BlockList + templist
        print("Fetched the {}".format(i))
    _BlockList = set(_BlockList)-set(_DistrictList)
    return _BlockList


In [38]:
import time
import os


def save_file(data, filename, path=os.getcwd(), filetype="txt"):
    """

    :param data:
    :param filename:
    :param path:
    :param filetype:
    :return:
    """
    if filetype == 'txt':
        with open("{0}/{1}".format(path, filename), "w") as f:
            for i in data:
                f.write("{}\n".format(i))



In [34]:
savefile(getblocklistofacity(OriginURL),"BlockList.txt")

Fetched the https://sh.lianjia.com/chengjiao/
Fetched the https://sh.lianjia.com/chengjiao/pudong/
Fetched the https://sh.lianjia.com/chengjiao/minhang/
Fetched the https://sh.lianjia.com/chengjiao/baoshan/
Fetched the https://sh.lianjia.com/chengjiao/xuhui/
Fetched the https://sh.lianjia.com/chengjiao/putuo/
Fetched the https://sh.lianjia.com/chengjiao/yangpu/
Fetched the https://sh.lianjia.com/chengjiao/changning/
Fetched the https://sh.lianjia.com/chengjiao/songjiang/
Fetched the https://sh.lianjia.com/chengjiao/jiading/
Fetched the https://sh.lianjia.com/chengjiao/huangpu/
Fetched the https://sh.lianjia.com/chengjiao/jingan/
Fetched the https://sh.lianjia.com/chengjiao/zhabei/
Fetched the https://sh.lianjia.com/chengjiao/hongkou/
Fetched the https://sh.lianjia.com/chengjiao/qingpu/
Fetched the https://sh.lianjia.com/chengjiao/fengxian/
Fetched the https://sh.lianjia.com/chengjiao/jinshan/
Fetched the https://sh.lianjia.com/chengjiao/chongming/
Fetched the https://sh.lianjia.com/che

In [77]:
import json
import time
import hashlib
import requests

group_type = ("district",     #行政区
              "bizcircle",    #商圈
              "community"     #居民小区
             )

b= {"city_id": "310000",
     "group_type": "district",
     "max_lat": "31.304485",
     "min_lat": "31.067068",
     "max_lng": "121.588912",
     "min_lng": "121.372168",
     }

def get_md5(txt):
    """md5加密函数"""
    m = hashlib.md5()
    m.update(txt.encode('utf-8'))
    return m.hexdigest()

def get_line_site(url):
    """请求链接"""
    headers = {'Accept': '*/*',
               'Accept-Encoding': 'gzip, deflate, br',
               'Accept-Language': 'zh-CN,zh;q=0.9',
               'Connection': 'keep-alive',
               'Host': 'ajax.lianjia.com',
               'Referer': 'https://gz.lianjia.com/ditu/',
               'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 '
                             '(KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'}
    r = requests.get(url, headers=headers)
    return r

def get_url(bbb):
    """拼接链接"""
    url ="https://ajax.lianjia.com/map/search/ershoufang/?city_id={city_id}&group_type={group_type}&" \
         "max_lat={max_lat}&min_lat={min_lat}&max_lng={max_lng}&min_lng={min_lng}&" \
         "filters=%7B%7D&request_ts={request_ts}&source=ljpc&" \
         "authorization={authorization}"
    request_ts = int(time.time() * 1000)
    md5_data = "vfkpbin1ix2rb88gfjebs0f60cbvhedlcity_id={city_id}group_type={group_type}max_lat={max_lat}" \
             "max_lng={max_lng}min_lat={min_lat}min_lng={min_lng}request_ts={request_ts}".format(
        city_id=bbb["city_id"],
        group_type=bbb["group_type"],
        max_lat=bbb["max_lat"],
        max_lng=bbb["max_lng"],
        min_lat=bbb["min_lat"],
        min_lng=bbb["min_lng"],
        request_ts=request_ts)
    authorization = get_md5(md5_data)
    url = url.format(request_ts=request_ts, authorization=authorization,city_id=bbb["city_id"],
        group_type=bbb["group_type"],
        max_lat=bbb["max_lat"],
        max_lng=bbb["max_lng"],
        min_lat=bbb["min_lat"],
        min_lng=bbb["min_lng"])
    return url

def getres(bb):
    line_url = get_url(bb)
    res = get_line_site(line_url)
    items = res.json()['data']
    return items is None

getres(b)


False

39

In [66]:
import json

ak = "dASz7ubuSpHidP1oQWKuAK3q"
def getCor(city):
    url = "http://api.map.baidu.com/geocoder/v2/?address={city}&output=json&ak={ak}".format(city="北京",ak=ak)
    res = requests.get(url).json()["result"]["location"]
    return res

def getCode(city):
    Cor = getCor(city)
    url2 = "http://api.map.baidu.com/geocoder/v2/?location={lat},{lng}&output=json&pois=1&ak={ak}".format(lat=Cor["lat"], lng=Cor["lng"],ak=ak)
    print(url2)
    res = requests.get(url2).json()
    return res

getCor("北京")
    

{'lng': 116.39564503787867, 'lat': 39.92998577808024}

In [28]:
res = requests.get("https://sh.lianjia.com/city/")


In [31]:
from bs4 import BeautifulSoup 
work = BeautifulSoup(res.text, "lxml")

In [49]:
import re
res = set([(i["href"], i.string) for i in work.find_all("a",href=re.compile(r"(?<=https://)\w+(?=\.lianjia.com/)"),target=False)][:-1])

In [48]:
import pandas as pd
citycodemapping = pd.read_csv(r"C:\Users\wenpeiyu\Desktop\citycodemapping.csv")
citycodemapping

,city,citycode
0,南充,511300
1,资阳,512000
2,广元,510800
3,成都,510100
4,重庆,500000
5,达州,511700
6,凉山彝族自治州,513400
7,巴中,511900
8,德阳,510600
9,广安,511600


In [57]:
dfcitylist = pd.DataFrame({
    "city": [i[1] for i in res],
    "cityurl": [i[0] for i in res]})
dfcitylist

,city,cityurl
0,天津,https://tj.lianjia.com/
1,南京,https://nj.lianjia.com/
2,沈阳,https://sy.lianjia.com/
3,济南,https://jn.lianjia.com/
4,烟台,https://yt.lianjia.com/
5,合肥,https://hf.lianjia.com/
6,廊坊,https://lf.lianjia.com/
7,重庆,https://cq.lianjia.com/
8,石家庄,https://sjz.lianjia.com/
9,厦门,https://xm.lianjia.com/


In [56]:
list(res)[0][0]

'https://tj.lianjia.com/'

In [59]:
dfCityList = pd.merge(dfcitylist,citycodemapping,how="inner",on=["city"] )

In [64]:
[i for i in dfCityList.city

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
           dtype='int64')

In [70]:
getres(dfCityList

SyntaxError: invalid syntax (<ipython-input-70-d74f754d9bea>, line 1)